In [1]:
# !pip install great-expectations==0.17.23 pandas==2.1.4
# !pip install hopsworks
# !pip install "hopsworks[python]"

In [2]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
# import os
# os.environ["FS_API_KEY"] = "your_actual_api_key"
# os.environ["FS_PROJECT_NAME"] = "your_project_name"

In [4]:
import os
os.environ["FS_API_KEY"] = "bfZD5E0Cqo9uuxGI.cl2n86qlQIwutJJ7HryOdp32jjryfoRUyd0xdVxt2YCLY6YJ5PkudygHlCVGeEe8"
os.environ["FS_PROJECT_NAME"] = "MSc_MLOps"

In [ ]:
import sys
from pathlib import Path

# Add the src folder to sys.path
project_root = Path().resolve().parent  # from notebooks/, this is the root dir
src_path = project_root / "src"
sys.path.append(str(src_path))

# Import the test_data module from the package
from fraud_project.pipelines.ingestion.nodes import ingestion
import pandas as pd

[06/19/25 20:05:12] INFO     Using                                                                  __init__.py:272
                             '/Users/mgalao/Library/CloudStorage/OneDrive-NOVAIMS/MSc_DSAA-DS/2nd                  
                             Semester/Q4 Machine Learning                                                          
                             Operations/machine-learning-operations-project/venv/lib/python3.12/sit                
                             e-packages/kedro/framework/project/rich_logging.yml' as logging                       
                             configuration.                                                                        

[06/19/25 20:05:13] WARNING  <frozen importlib._bootstrap>:488: DeprecationWarning: Type            ]8;id=212362;file:///opt/anaconda3/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=965974;file:///opt/anaconda3/lib/python3.12/warnings.py#112\112]8;;\
                             google._upb._message.MessageMapContainer uses PyType_Spec with a                      
                             metaclass that has custom tp_new. This is deprecated and will no                      
                             longer be allowed in Python 3.14.                                                     
                                                                                                                   

                    WARNING  <frozen importlib._bootstrap>:488: DeprecationWarning: Type            ]8;id=108244;file:///opt/anaconda3/lib/python3.12/warnings.py\warnings.py]8;;\:]8;id=149086;file:///opt/anaconda3/lib/python3.12/warnings.py#112\112]8;;\
                             google._upb._message.ScalarMapContainer uses PyType_Spec with a                       
                             metaclass that has custom tp_new. This is deprecated and will no                      
                             longer be allowed in Python 3.14.                                                     
                                                                                                                   

# With Fraud DF

In [6]:
df = pd.read_csv('../data/01_raw/data_v1.csv')
df.head()

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,...,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud,merch_zipcode
0,2019-05-04 11:57:04,4586260469584,fraud_Kerluke Inc,misc_net,5.96,Melody,Thompson,F,0362 Anderson Wall,Mound City,...,-95.2138,1631,Architect,1953-01-20,3d21bce7967838c3988cfe0f7fca878a,1336132624,41.024651,-94.428240,0,50842.0
1,2019-12-14 08:55:21,4900628639996,fraud_Rempel PLC,grocery_net,70.66,Michael,Johnson,M,094 Owens Underpass,Norwalk,...,-118.0818,105549,Firefighter,1973-09-22,fda7712b4bbcaab36afded37ab55047f,1355475321,33.808771,-118.031888,0,90630.0
2,2019-03-30 05:21:33,676118385837,fraud_Rodriguez Group,gas_transport,50.92,Katelyn,Wise,F,674 Maureen Summit Apt. 276,Scotts Mills,...,-122.6187,1252,"Engineer, petroleum",1937-02-06,59161e0002642934974c1ae98bfa1f55,1333084893,44.561034,-123.281803,0,97331.0
3,2019-09-19 07:09:46,3596357274378601,fraud_Doyle Ltd,grocery_pos,71.68,David,Everett,M,4138 David Fall,Morrisdale,...,-78.2357,3688,Advice worker,1973-05-27,f487a7098c0bd4d45f710be1745c4acb,1348038586,41.612825,-78.316893,0,15834.0
4,2019-02-04 20:37:44,6011542681743618,fraud_Leffler-Goldner,personal_care,29.17,Emily,Hall,F,8851 Reese Neck,Basye,...,-78.7776,863,"Engineer, mining",1972-08-09,c2ed76f03cce8a6b362729a5a23f01c2,1328387864,39.387521,-79.674956,0,26444.0


In [7]:
# Load the configuration parameters
from pathlib import Path
from kedro.config import OmegaConfigLoader

conf_path = Path('../conf').resolve()
conf_loader = OmegaConfigLoader(conf_source=conf_path)
parameters = conf_loader["parameters"]

print(parameters)

{'target_column': 'is_fraud', 'to_feature_store': True, 'test_fraction': 0.3, 'random_state': 2021, 'run_name': 'full_project', 'use_feature_selection': False, 'feature_selection': 'rfe', 'model_name': 'fraud', 'baseline_model_params': {'n_estimators': 100, 'max_depth': 6, 'max_features': 20, 'random_state': 2021}, 'hyperparameters': {'RandomForestClassifier': {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 20], 'max_features': [0.5, 0.7, 1], 'random_state': [2021]}, 'GradientBoostingClassifier': {'n_estimators': [100, 200, 300], 'max_depth': [5, 10, 20], 'max_features': [0.5, 0.7, 1], 'random_state': [2021]}}}


In [8]:
df.sample(100)
df_processed = ingestion(df, parameters)


Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224874
Column 'trans_num' type: <class 'pandas.core.series.Series'>
Column 'datetime' type: <class 'pandas.core.series.Series'>
Column 'amt' type: <class 'pandas.core.series.Series'>
Column 'lat' type: <class 'pandas.core.series.Series'>
Column 'long' type: <class 'pandas.core.series.Series'>
Column 'city_pop' type: <class 'pandas.core.series.Series'>
Column 'unix_time' type: <class 'pandas.core.series.Series'>
Column 'merch_lat' type: <class 'pandas.core.series.Series'>
Column 'merch_long' type: <class 'pandas.core.series.Series'>
Column 'age' type: <class 'pandas.core.series.Series'>
Validation failed.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1224874/fs/1212504/fg/1484333


Uploading Dataframe: 100.00% |██████████| Rows 90767/90767 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: numerical_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224874/jobs/named/numerical_features_1_offline_fg_materialization/executions
Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224874
Column 'trans_num' type: <class 'pandas.core.series.Series'>
Column 'datetime' type: <class 'pandas.core.series.Series'>
Column 'cc_num' type: <class 'pandas.core.series.Series'>
Column 'merchant' type: <class 'pandas.core.series.Series'>
Column 'category' type: <class 'pandas.core.series.Series'>
Column 'first' type: <class 'pandas.core.series.Series'>
Column 'last' type: <class 'pandas.core.series.Series'>
Column 'gender' type: <class 'pandas.core.series.Series'>
Column 'street' type: <class 'pandas.core.series.Series'>
Column 'city' type: <class 'pandas.core.series.Series'>
Column 'state' type: <class 'pandas.core.series.Series'>
Column 'zip' type: <class 'p

Uploading Dataframe: 100.00% |██████████| Rows 90767/90767 | Elapsed Time: 00:03 | Remaining Time: 00:00


Launching job: categorical_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224874/jobs/named/categorical_features_1_offline_fg_materialization/executions
Connection closed.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1224874
Column 'trans_num' type: <class 'pandas.core.series.Series'>
Column 'datetime' type: <class 'pandas.core.series.Series'>
Column 'is_fraud' type: <class 'pandas.core.series.Series'>
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1224874/fs/1212504/fg/1484335
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1224874/fs/1212504/fg/1484335


Uploading Dataframe: 100.00% |██████████| Rows 90767/90767 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: target_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1224874/jobs/named/target_features_1_offline_fg_materialization/executions
Statistics Job started successfully, you can follow the progress at 
http://c.app.hopsworks.ai/p/1224874/jobs/named/target_features_1_compute_stats_19062025191431/executions


# With fraud DF

In [ ]:
df_fraud_data_additional = pd.read_csv('../data/01_raw/bank_data_additional.csv')
df_fraud_data_additional.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [82]:
test_data(df_fraud_data_additional)

Calculating Metrics:   0%|          | 0/18 [00:00<?, ?it/s]

,Success,Expectation Type,Column,Column Pair,Max Value,Min Value,Element Count,Unexpected Count,Unexpected Percent,Value Set,Unexpected Value,Observed Value
0,True,expect_column_distinct_values_to_be_in_set,marital,"(, )",,,,,,"[married, single, divorced]",[],"[divorced, married, single]"
1,False,expect_column_values_to_be_between,balance,"(, )",105000,0,45211,3766,8.329831,,[],
2,True,expect_column_values_to_be_between,age,"(, )",100,18,45211,0,0.0,,[],
